In [1]:
import pandas as pd

In [19]:
import ast

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
rating_data = pd.read_csv('./archive/ratings_small.csv')
movie_data = pd.read_csv('./archive/movies_metadata.csv')

C:\Users\komac\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
rating_data.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [4]:
rating_data.shape

(100004, 4)

In [5]:
movie_data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [6]:
movie_data.shape

(45466, 24)

In [7]:
rating_data.drop('timestamp', axis = 1, inplace=True)
rating_data.head(2)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0


In [8]:
movie_data = movie_data[['id', 'genres', 'title']]

In [9]:
# 지정해서 컬럼명 변경
movie_data = movie_data.rename(columns={'id':'movieId'})
movie_data.head(2)

,movieId,genres,title
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji


In [10]:
rating_data['movieId'][1]

1029

In [11]:
movie_data['movieId'][1]

'8844'

In [12]:
for i in range(len(movie_data)):
    if '-' in movie_data.loc[i,'movieId']:
        print(i, movie_data.loc[i,'movieId'])

19730 1997-08-20
29503 2012-09-29
35587 2014-01-01


In [13]:
movie_data.loc[19730,'movieId'] = 0
movie_data.loc[29503,'movieId'] = 0
movie_data.loc[35587,'movieId'] = 0

In [14]:
movie_data['movieId'] = movie_data['movieId'].apply(pd.to_numeric)

In [15]:
movie_data['movieId'][1]

8844

In [16]:
user_movie_rating = pd.merge(rating_data, movie_data, on = 'movieId')

In [17]:
user_movie_rating.head(3)

,userId,movieId,rating,genres,title
0,1,1371,2.5,"[{'id': 18, 'name': 'Drama'}]",Rocky III
1,4,1371,4.0,"[{'id': 18, 'name': 'Drama'}]",Rocky III
2,7,1371,3.0,"[{'id': 18, 'name': 'Drama'}]",Rocky III


In [20]:
user_movie_rating['genres'] = user_movie_rating['genres'].apply(ast.literal_eval)

In [21]:
user_movie_rating.head(3)

,userId,movieId,rating,genres,title
0,1,1371,2.5,"[{'id': 18, 'name': 'Drama'}]",Rocky III
1,4,1371,4.0,"[{'id': 18, 'name': 'Drama'}]",Rocky III
2,7,1371,3.0,"[{'id': 18, 'name': 'Drama'}]",Rocky III


In [22]:
user_movie_rating['genres'] = user_movie_rating['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [23]:
user_movie_rating.head(3)

,userId,movieId,rating,genres,title
0,1,1371,2.5,Drama,Rocky III
1,4,1371,4.0,Drama,Rocky III
2,7,1371,3.0,Drama,Rocky III


### 전처리 끝

In [30]:
movie_user_rating = user_movie_rating.pivot_table('rating', index='title', columns='userId')
user_movie_rating = user_movie_rating.pivot_table('rating', index='userId', columns='title')

In [33]:
user_movie_rating.head()

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
movie_user_rating.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Gator Bait,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Twas the Night Before Christmas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...And God Created Woman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00 Schneider - Jagd auf Nihil Baxter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
movie_user_rating.fillna(0, inplace = True)
movie_user_rating.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Gator Bait,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Twas the Night Before Christmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


아이템 기반 협업 필터링 추천 시스템은 유사한 아이템끼리 추천해줌

유사한 아이템이라고 하면 -> 평점이 비슷한 아이템
이미 평점은 data로 들어가 있기 때문에 이 상태에서 코사인 유사도를 구하기

In [36]:
item_based_collabor = cosine_similarity(movie_user_rating)
item_based_collabor

array([[1.        , 0.        , 0.51370361, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.03818018, 0.        ,
        0.        ],
       [0.51370361, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03818018, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [37]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = movie_user_rating.index, columns = movie_user_rating.index)

In [38]:
item_based_collabor.head()

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,1.000000,0.0,0.513704,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.118456,0.000000,0.000000,0.0,0.0
'Gator Bait,0.000000,1.0,0.000000,0.000000,0.000000,0.207390,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.182018,0.000000,0.038180,0.0,0.0
'Twas the Night Before Christmas,0.513704,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...And God Created Woman,0.000000,0.0,0.000000,1.000000,0.789352,0.345651,0.0,0.0,0.0,0.0,...,0.0,1.000000,0.631169,0.813733,0.339227,0.130013,1.000000,0.000000,0.0,0.0
00 Schneider - Jagd auf Nihil Baxter,0.000000,0.0,0.000000,0.789352,1.000000,0.272840,0.0,0.0,0.0,0.0,...,0.0,0.789352,0.498214,0.642322,0.267769,0.102626,0.789352,0.210963,0.0,0.0


In [39]:
def get_item_based_collabor(title):
    return item_based_collabor[title].sort_values(ascending=False)[:6]

In [41]:
get_item_based_collabor('!Women Art Revolution')

title
!Women Art Revolution    1.000000
Premium Rush             0.996097
Shadow Run               0.759257
The Eleventh Victim      0.759257
The English Patient      0.650791
Stick                    0.650791
Name: !Women Art Revolution, dtype: float64